In [1]:
import pandas as pd
import numpy as np
import math
from collections import defaultdict
import queue
import heapq

In [2]:
#this block is to declare the necessary data structures and then take the whole graph as the input
#taking the number of nodes and then the coordinate as the input
n = -1                        #this is for the number of nodes
nodes = dict()                    #storing each node and there coordinate


file = open ("in.txt" , "r")

#this is for the number of nodes n
words = file.readline()
words = words.strip().split()
n = int (words[0])

#this is for reading the nodes and then there coordinates
#each node and there and there coordinate are stored in the dictionary
for _ in range(n):
    line = file.readline()
    words = line.strip().split()
    temp , coordinated = int(words[0]) , (int(words[1]) , int (words[2]))
    temp -= 1
    nodes[temp] = coordinated
    
#taking the number of edges
words = file.readline().strip().split()
edges = int(words[0])

#storing the edges in the adjacency list
graph = [[] for _ in range(n)]
for _ in range(edges):
    
    #taking each edge as the input
    line = file.readline()
    words = line.strip().split()
    x , y, weight = int (words[0]) - 1 , int (words[1]) - 1 , float(words[2])

    #storing each edge in the adjacency list
    graph[x].append( (y , weight))
    graph[y].append( (x , weight))
    
#taking the start state
start = file.readline().strip().split()
start = int(start[0]) - 1

goal = file.readline().strip().split()
goal = int(goal[0]) - 1
    
file.close()
    
    

In [3]:
#this block is to print all the informations and then 
print("number of nodes : " , n)
print("Nodes with there coordinates : ")
print(nodes)

print("Graph : ")
for i in range(n):
    print(i+1, end = " ")
    print(graph[i])


print("Start state : " , start)
print("Goal state : " , goal)

number of nodes :  17
Nodes with there coordinates : 
{0: (0, 0), 1: (0, 1), 2: (0, 2), 3: (0, 3), 4: (1, 0), 5: (1, 1), 6: (2, 0), 7: (3, 1), 8: (3, 2), 9: (3, 3), 10: (4, 1), 11: (5, 0), 12: (5, 1), 13: (5, 2), 14: (5, 3), 15: (6, 1), 16: (6, 2)}
Graph : 
1 [(4, 1.0), (1, 2.5)]
2 [(0, 2.5), (2, 2.5)]
3 [(1, 2.5), (8, 6.0), (3, 3.0), (9, 2.5)]
4 [(2, 3.0), (9, 3.0)]
5 [(0, 1.0), (5, 6.0)]
6 [(4, 6.0), (6, 4.0), (7, 6.0), (8, 10.0)]
7 [(5, 4.0), (11, 10.0)]
8 [(5, 6.0), (10, 6.0)]
9 [(2, 6.0), (5, 10.0), (13, 15.0), (14, 20.0)]
10 [(2, 2.5), (3, 3.0), (14, 10.0)]
11 [(7, 6.0), (12, 4.0)]
12 [(6, 10.0), (12, 2.0)]
13 [(10, 4.0), (11, 2.0), (13, 12.0), (15, 11.0)]
14 [(8, 15.0), (12, 12.0), (14, 5.0), (16, 2.5)]
15 [(8, 20.0), (9, 10.0), (13, 5.0)]
16 [(12, 11.0), (16, 1.0)]
17 [(13, 2.5), (15, 1.0)]
Start state :  0
Goal state :  16


In [4]:
#using the same heuristic as that of the euclidean 
#but here we are going to take the vehicle that is the fastest and calculate time from it
def euclidean_time (x , speed):
    return math.sqrt((x[0] - nodes[goal][0]) ** 2 + (x[1] - nodes[goal][1]) ** 2) / speed

In [5]:
#this function is going to take one argument of the number of congestion
#this is not going to take care of the cost 
#just find the shortest path without taking into account of the budget
def solve(congestion ):
    #1 implies the full congestion
    speed_cycle = 25.0
    if (congestion == 1):
        speed_bus = 100.0
    elif (congestion == 2):
        speed_bus = 37.5
    else:
        speed_bus = 10.0
       
    #this is computed in advance to compute the heuristic
    speed = max (speed_bus , speed_cycle)
    
    d = dict()                  #to store the state of each nodes
                                #first element will store the parent
                                #second will store the cost till now
                                #third will store the heuristic cost
                                    
    q = []                      #this is for the queue
    heapq.heapify(q)
    
    explored = set()
    
    counter_queue = 0           #this is to break the counter
    #initialise all the data structure
    for i in range(n):                              #number of states is now twice
        d[i] = (-1 , math.inf , euclidean_time(nodes[i] , speed))
        
    #this is for the start node
    d[start] = (-1 , 0 , euclidean_time(nodes[start] , speed))
    heapq.heappush (q , (d[start][1] + d[start][2] ,counter_queue , start))      #one more element is added to add the budget
    
    counter = 0
    ans = False
    while (len(q) >= 0 and ans == False):
        
        counter += 1
        print("Number of steps : " , counter)
    
        #popping the element
        dist , waster ,current = heapq.heappop(q)
        
            
        #checking for the goal state
        if (current == goal):
            ans = True
            break
            
        explored.add(current)
        
        #for the lazy deletion
        if (dist > d[current][1] + d[current][2]):
            continue
            
        #main code 
        #extracting each edge and then updating the queue
        for v in graph[current]:
            vertex = v[0]
            weight = v[1]
            if (vertex not in explored):
                if (weight <= 3):                  #this meant it is only for the cycle
                    if (d[vertex][1] > d[current][1] + weight/speed_cycle ):
                        d[vertex] = (current , d[current][1] + weight/speed_cycle , euclidean_time(nodes[vertex] , speed))
                        counter_queue += 1
                        heapq.heappush(q , (d[vertex][1] + d[vertex][2] , counter_queue  ,vertex))
                
                else:                                  #this meant both the bus and the cycle can take the action
                    if (d[vertex][1] > d[current][1] + weight/speed_bus ):
                        d[vertex] = (current , d[current][1] + weight/speed_bus , euclidean_time(nodes[vertex] , speed))
                        counter_queue += 1
                        heapq.heappush(q , (d[vertex][1] + d[vertex][2] , counter_queue , vertex))
                        
                    if (d[vertex][1] > d[current][1] + weight/speed_cycle ):
                        d[vertex] = (current , d[current][1] + weight/speed_cycle , euclidean_time(nodes[vertex] , speed))
                        counter_queue += 1
                        heapq.heappush(q , (d[vertex][1] + d[vertex][2] , counter_queue , vertex))
                    
                    
        
    if (ans == False):
        print("No solution")
        return
    
    current = goal
    path = []
    while (current != -1):
        path.insert(0 , current)
        current = d[current][0]
    
    print("The path is : ")
    for i in path:
        print(i+1 , end = " ")
    
    print("\nTotal cost (in terms of timer , in minutes) : " , end = " ")
    print(60 * d[goal][1])
        
        

In [6]:
solve(1)            #this is with full congestion

Number of steps :  1
Number of steps :  2
Number of steps :  3
Number of steps :  4
Number of steps :  5
Number of steps :  6
Number of steps :  7
Number of steps :  8
Number of steps :  9
Number of steps :  10
Number of steps :  11
Number of steps :  12
Number of steps :  13
Number of steps :  14
Number of steps :  15
Number of steps :  16
The path is : 
1 5 6 8 11 13 16 17 
Total cost (in terms of timer , in minutes) :  24.599999999999998


In [7]:
solve(2)                  #half congestion

Number of steps :  1
Number of steps :  2
Number of steps :  3
Number of steps :  4
Number of steps :  5
Number of steps :  6
Number of steps :  7
Number of steps :  8
Number of steps :  9
Number of steps :  10
Number of steps :  11
Number of steps :  12
Number of steps :  13
Number of steps :  14
Number of steps :  15
Number of steps :  16
Number of steps :  17
Number of steps :  18
The path is : 
1 2 3 10 15 14 17 
Total cost (in terms of timer , in minutes) :  47.99999999999999


In [8]:
solve(3)                            #this is for the full congestion

Number of steps :  1
Number of steps :  2
Number of steps :  3
Number of steps :  4
Number of steps :  5
Number of steps :  6
Number of steps :  7
Number of steps :  8
Number of steps :  9
Number of steps :  10
Number of steps :  11
Number of steps :  12
Number of steps :  13
Number of steps :  14
Number of steps :  15
Number of steps :  16
Number of steps :  17
Number of steps :  18
Number of steps :  19
Number of steps :  20
The path is : 
1 2 3 10 15 14 17 
Total cost (in terms of timer , in minutes) :  60.000000000000014


In [9]:
#this function is going to take one argument of the number of congestion
def solve_using_budget(congestion , budget , cost ):
    #1 implies the full congestion
    speed_cycle = 25.0
    if (congestion == 1):
        speed_bus = 100.0
    elif (congestion == 2):
        speed_bus = 37.5
    else:
        speed_bus = 10.0
       
    #this is computed in advance to compute the heuristic
    speed = max (speed_bus , speed_cycle)
    
    
    d = dict()                  #to store the state of each nodes
                                #first element will store the parent
                                #second will store the cost till now
                                #third will store the heuristic cost
                                #each node is now defined with the help of the vertex and the current budget
    
    q = []                      #this is for the queue
    heapq.heapify(q)
    
    explored = set()
    frontier = []
    
    
    counter_queue = 0           #this is to break the counter
    #initialise all the data structure
    
    #this is for the start node
    #initialising the priority queue, state and frontier
    d[(start , budget* 100)] = ((-1,-1) , 0 , euclidean_time(nodes[start] , speed))
    heapq.heappush (q , (d[(start , budget*100)][1] + d[(start , budget*100)][2] , 100 * budget ,counter_queue  , start))      #one more element is added to add the budget
    frontier.append((start , budget * 100))
    
    #to keep track of wether the solution is found or not
    ans = False
    final_money_left = -1
    while (len(q) >= 0 and ans == False):
        
        #popping the element
        dist ,  money_left, waster ,current = heapq.heappop(q)
        frontier.remove((current , money_left))
        
        
        #if money left is less than 0 then do not expand that node    
        if (money_left < 0):
            continue
         
        #checking for the goal state
        if (current == goal):
            final_money_left = money_left
            ans = True
            break
         
        #adding the node to the set of the explored state
        explored.add((current, money_left))
        
        #for the lazy deletion
        if (dist > d[(current, money_left)][1] + d[ (current, money_left)][2]):
            continue
        
        #for converting it to the real money
        money_temp = money_left / 100
        
        #main code 
        #extracting each edge and then updating the queue
        for v in graph[current]:
               
            #only one action possible for the cycle
            vertex = v[0]
            weight = v[1]
            budget_left = money_left
            state = (vertex , budget_left)
            
            #for the cycle part 
            #The action here is to take the cycle
            if (state not in explored and state not in frontier):
                d[state] = ((current, money_left) , d[(current, money_left)][1] + weight/speed_cycle , euclidean_time(nodes[start] , speed))
                counter_queue += 1
                heapq.heappush(q , (d[state][1] + d[state][2] , budget_left , counter_queue , vertex))
                frontier.append( state )
                
            elif state in frontier:
                if (d[state][1] > d[(current, money_left)][1] + weight/speed_cycle ):
                    d[state] = ((current, money_left) , d[(current, money_left)][1] + weight/speed_cycle , euclidean_time(nodes[start] , speed))
                    counter_queue += 1
                    heapq.heappush(q , (d[state][1] + d[state][2] , budget_left , counter_queue ,  vertex))
                    frontier.append( state )
            
            #the action here is to take the bus
            #calculating the cost and then checking if the bus can be taken or not
            budget_left = int ((money_temp - (weight / speed_bus) * cost) * 100)
            if (weight > 3 and budget_left > 0):  #now both the actions are allowed, so bus is also allowed
                state = (vertex , budget_left)
                if (state not in explored and state not in frontier):
                    d[state] = ((current, money_left), d[(current, money_left)][1] + weight/speed_bus , euclidean_time(nodes[start] , speed))
                    counter_queue += 1
                    heapq.heappush(q , (d[state][1] + d[state][2] , budget_left, counter_queue ,  vertex))
                    frontier.append( state )
                
                elif state in frontier:
                    if (d[state][1] > d[(current, money_left)][1] + weight/speed_bus ):
                        d[state] = ((current, money_left), d[(current, money_left)][1] + weight/speed_bus , euclidean_time(nodes[start] , speed))
                        counter_queue += 1
                        heapq.heappush(q , (d[state][1] + d[state][2] , budget_left ,counter_queue ,  vertex))
                        frontier.append( state )
    
    #if the solution is not found
    if (ans == False):
        print("No solution")
        return
    
    #storing the path in path list and then printing the path
    current = (goal , final_money_left)
    path = []
    while (current[0] != start):
        path.insert(0 , current[0])
        current = d[current][0]
    
    #printing the path
    path.insert(0 , start)
    print("The path is : ")
    for i in path:
        print(i+1 , end = " ")
    
    #printing the path
    current = (goal , final_money_left)
    print("\nTotal cost (in terms of timer , in minutes) : " , end = " ")
    print(60 * d[current][1])
    print( "budget " , budget - final_money_left/100)
        
        

In [10]:
solve_using_budget(2 , 1000, 100)

The path is : 
1 2 3 10 15 14 17 
Total cost (in terms of timer , in minutes) :  47.99999999999999
budget  40.00999999999999


In [11]:
solve_using_budget(3 , 1000 , 1000)

The path is : 
1 2 3 10 15 14 17 
Total cost (in terms of timer , in minutes) :  60.000000000000014
budget  0.0


In [12]:
solve_using_budget(1 , 1000 , 1000)

The path is : 
1 5 6 8 11 13 16 17 
Total cost (in terms of timer , in minutes) :  24.599999999999998
budget  330.0


In [13]:
solve_using_budget(1 , 1000 , 10000)

The path is : 
1 2 3 10 15 14 17 
Total cost (in terms of timer , in minutes) :  51.00000000000001
budget  500.0


In [14]:
solve_using_budget(3 , 0 , 1)

The path is : 
1 2 3 10 15 14 17 
Total cost (in terms of timer , in minutes) :  60.000000000000014
budget  0.0
